In [2]:
## Importing Qiskit libraries
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, AncillaRegister, Aer, execute, assemble
from qiskit_aer import AerSimulator
from qiskit.circuit import Gate
from qiskit.visualization import plot_histogram, circuit_drawer
import matplotlib.pyplot as plt
from qiskit.quantum_info import Operator
from qiskit.circuit.library import MCMT, RYGate, RXGate, PhaseGate
import numpy as np
np.set_printoptions(threshold=np.inf)
import math
from typing import Union
from numpy import linalg
from scipy.linalg import expm
import pickle

In [3]:
## Settings
d = 1 # of dimensions
M = 1 # of registers (rough estimate for M is the number of particles in the final state, but M should be set so that p_success can be maximized)
n = 1 # of particles in the initial state
N_abs = 1 # of modes for momenta
m = 1 # mass of particles of projectile


nqubits = N_abs * d + d + 1
N_s = 2 ** (N_abs + 1) # size of the lattice per one dimension
V = N_s ** d # volume
N = N_abs * d + d + 1 # of qubits per one particle (magnitude of momentum + sign + occupation)
s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.

In [4]:
I = np.array([[1, 0], [0, 1]])
H = np.array([[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), -1/np.sqrt(2)]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

In [5]:
#初期状態|0000・・・0>を準備する。
def StateZeros(nqubits):
    State = np.zeros(2**nqubits)
    State[0]=1
    return State

In [6]:
def makeGate(N, nqubit, Gate): # action of Gate to n'th qubit (n = 0...N)
    if nqubit == 0:
        tmp = Gate
    else: tmp = I
    for i in range(N):
        B_tmp = np.zeros((2 ** (i + 2), 2 ** (i + 2)))
        if nqubit == i + 1:
            B_tmp = np.kron(Gate, tmp)
        else: B_tmp = np.kron(I, tmp)
        tmp = B_tmp
    return tmp

In [7]:
# makeGate(N, (N_abs + 1) * d, X)
# makeGate(1, 1, H)

In [8]:
def transPosition2Momentum(qmom, sign): # if sign is True, we have to transform qmom to qposition
    if sign == 'position':
        qmom = qmom - (N / 2)
    return qmom

In [9]:
# if dagger is True, make an annihilation operator, if False, an creation operator
# ireg means the number of the register acted
# qmom means that the momentum with sign
def annahilationCreationOpPerReg(qmom, ireg, dagger, N, M, sign):
    qmom = transPosition2Momentum(qmom, sign)
    if qmom > 0:
        qrow = int(qmom - 1/2 + 2 ** (N-1) + 2 ** N_abs)
    elif qmom < 0:
        qrow = int(np.abs(qmom + 1/2) + 2 ** (N-1))
    tmp_op = np.zeros((2 ** N, 2 ** N))
    if dagger == False:
        tmp_op[0][qrow] = 1
    else:
        tmp_op[qrow][0] = 1
    identity = np.eye(2 ** N)
    if ireg == 0:
        tmp = tmp_op
    else: tmp = identity
    # print(tmp)
    for i in range(M-1):
        # print(i)
        B_tmp = np.zeros((2 ** (i + 2), 2 ** (i + 2)))
        if ireg == i + 1:
            B_tmp = np.kron(tmp_op, tmp)
        else:
            B_tmp = np.kron(identity, tmp)
        tmp = B_tmp
    return tmp

In [31]:
a = annahilationCreationOpPerReg(1/2, 0, False, 3, 1, 'momentum')
b = annahilationCreationOpPerReg(3/2, 0, False, 3, 1, 'momentum')
a @ b

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [10]:
def annahilationCreationOp(qmom, dagger, sign):
    tmp_op = np.zeros((2 ** (N * M), 2 ** (N * M)))
    # print(tmp_op.size)
    for ireg in range(M):
        # print(annahilationCreationOpPerReg(qmom, ireg, dagger, N, M).size)
        tmp_op += annahilationCreationOpPerReg(qmom, ireg, dagger, N, M, sign)
    return tmp_op * (1/np.sqrt(M))

In [466]:
M = 2
annahilationCreationOp(1/2, False, 'momentum')

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.70710678, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0. 

In [11]:
def changeBasis(N):
    tmp = np.eye(2 ** N)
    for i in range(2 ** (N - 2)):
        tmp[2 ** (N - 1) + i][2 ** (N - 1) + i] = 0
        tmp[2 ** (N - 1) + i][2 ** (N - 1) + 2 **(N - 2) - i - 1] = 1
    return tmp

In [12]:
import cmath

def dft_matrix(num):
    A = np.arange(num)
    B = A.reshape(1, -1)
    C = A.reshape(-1, 1)
    M = cmath.e**(-1j * 2 * cmath.pi * B * C  / num)
    return M * (1/np.sqrt(num))

In [13]:
def qft(N):
    return np.kron(np.eye(2), dft_matrix(2 ** (N - 1)))

In [14]:
def makeGatePerReg(Gate, M): # make tensor M products of Gate 
    tmp = Gate
    for i in range(M - 1):
        B_tmp = np.zeros((2 ** (N * i), 2 ** (N * i)))
        B_tmp = np.kron(Gate, tmp)
        tmp = B_tmp
    return tmp

In [432]:
# annahilationCreationOpPerReg(3/2, 0, False, 3, 2, 'momentum')
# annahilationCreationOp(1, True)

In [40]:
def sqij(qmom, ireg, jreg):
    zq = (1/2) * np.log(np.abs(qmom))
    A = annahilationCreationOpPerReg(qmom, ireg, True, N, M, 'momentum')
    B = annahilationCreationOpPerReg(-qmom, jreg, True, N, M, 'momentum')
    C = annahilationCreationOpPerReg(-qmom, jreg, False, N, M, 'momentum')
    D = annahilationCreationOpPerReg(qmom, ireg, False, N, M, 'momentum')
    ln_sq = -(zq/M) * (A @ B - C @ D)
    sq_ij = expm(ln_sq)
    return sq_ij

In [62]:
len(sqij(1/2, 0, 1))
list = []
for i in range(64):
    if sqij(1/2, 0, 1)[i][0] != 0:
        list.append(sqij(1/2, 0, 1)[i][0])
        print(i)
list

0
38


[0.985023376545766, 0.1724208446168214]

In [15]:
def squeezeOpPerMom(qmom):
    zq = (1/2) * np.log(np.abs(qmom))
    A = annahilationCreationOp(qmom, True, 'momentum')
    # print('A is', A)
    B = annahilationCreationOp(-qmom, True, 'momentum')
    # print('B is', B)
    C = annahilationCreationOp(-qmom, False, 'momentum')
    # print('C is', C)
    D = annahilationCreationOp(qmom, False, 'momentum')
    # print(linalg.norm(A @ B, 2))
    ln_sq = -zq * (A @ B - C @ D)
    sq = expm(ln_sq)
    return sq

In [63]:
def squeezeOp():
    sq = np.eye(2 ** (N * M))
    for q in range(N_abs + 1):
        # print(q)
        # print(squeezeOpPerMom(q+1/2))
        # print(squeezeOpPerMom(-(q+1/2)))
        sq @= squeezeOpPerMom(q + 1/2)
        # sq @= squeezeOpPerMom(-(q + 1/2))
    return sq

In [17]:
def phi(n):
    A = annahilationCreationOp(n, True, 'position')
    B = annahilationCreationOp(n, False, 'position')
    phiGate = (1/np.sqrt(2)) * (A + B)
    return phiGate

In [18]:
def intU(delta, lam):
    ln_UI = 0
    for ipos in range(N_abs * d + d):
        ln_UI += -1j * delta * (lam/24) * phi(ipos) @ phi(ipos) @ phi(ipos) @ phi(ipos)
    uI = expm(ln_UI)
    return uI

In [434]:
# intU(0.1, 0.1) @ intU(0.1, 0.1).conj().T

In [19]:
def addAncilla(mat, N_abs, n, d, s):
    free_num = (N_abs + math.log2(n)) * d
    bose_num = s + 1
    anc_eye_mat = np.eye(2 ** int(free_num + bose_num))
    return np.kron(anc_eye_mat, mat)

In [73]:
M = 2
N_abs = 1
d = 1
N = N_abs * d + d + 1
n = 1
s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.
tau_0 = 1 # Wigner delay
tau_I = 0 # length of time of interaction
delta = 1.5 # time step
coupling_lambda = 0.1
m_0 = 0.5 # chosen to represent a relevant enrgy scale in the weak coupling regime
tau = 0
m_t = m_0
lambda_t = 0

m_ren = 0
lambda_ren = 1
free_num = int((N_abs + math.log2(n)) * d)
bose_num = int(s + 1)
state = StateZeros(((N * M) + (free_num + bose_num)))
count = 0

while tau <= tau_0 + tau_I:
    print(count)
    # # Squeezing operation
    # S = addAncilla(squeezeOp(), N_abs, n, d, s)
    # state = np.dot(S, state)

    # Change basis
    C = addAncilla(makeGatePerReg(changeBasis(N), M), N_abs, n, d, s)
    state = np.dot(C, state)

    # QFT
    Q = addAncilla(makeGatePerReg(qft(N), M), N_abs, n, d, s)
    state = np.dot(Q, state)

    # # Interaction Hamiltonian
    # U_I = addAncilla(intU(delta, lambda_t), N_abs, n, d, s)
    # state = np.dot(U_I, state)

    # inverse QFT
    # Q = addAncilla(makeGatePerReg(qft(N), M), N_abs, n, d, s)
    # invQ = np.linalg.inv(Q)
    # state = np.dot(invQ, state)

    # # Change basis
    # C = addAncilla(makeGatePerReg(changeBasis(N), M), N_abs, n, d, s)
    # state = np.dot(C, state)
    count += 1

    tau += delta
    if tau < tau_0: # turn on interaction adiabatically
        lambda_t = lambda_ren * (tau / tau_0)
        m_t = m_0 * (1 - (tau / tau_0)) + m_ren * (tau / tau_0)
    elif tau_0 <= tau : # interaction completely
        m_t = m_ren
        lambda_t = lambda_ren

print(state)

# 結果を保存
result_file = 'calculated_result.pkl'
with open(result_file, 'wb') as f:
    pickle.dump(state, f)

0
[0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0. 